In [32]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Project').getOrCreate()
import pandas as pd
import itertools

In [33]:
columns= []
table_rows =[]
Decision =''
decision_rows=[]
table_header =[]
table_content =[]


In [34]:
#ask the user to enter the file name and then the load the file data
def readFile():
    is_Columns_found = False
    column_count_loc=0
    print('Enter FileName ')
    FileName = input("> ")
    print("Loading file {}".format(FileName))
    file = open(FileName, 'r')
    for row_Number, row in enumerate(file):
        row_Item = row.split()
        if(len(row_Item)==0):
            continue
        if(row_Item[0].startswith('<') or row_Item[0].startswith('!')):
            continue
        elif(is_Columns_found==False and row_Item[0].startswith('[')):
            columns = row_Item[1:-2]
            Decision=row_Item[-2]
            table_header=row_Item[1:-1]
            is_Columns_found=True
            print(is_Columns_found)
        else:
            table_rows.append(row_Item[0:-1])
            decision_rows.append(row_Item[-1])
            row_Item.append(row_Number+1)
            table_content.append(row_Item)
            #table_content.append(row_Number+1)
            column_count_loc+=1
    print('Printing Attributes : ',columns)
    print('Printing Decision : ',Decision)
    return column_count_loc,columns,table_rows,Decision,decision_rows,table_header,table_content


In [35]:
#Calling Read file function
column_count,columns,table_rows,Decision,decision_rows,table_header,table_content=readFile()

Enter FileName 
> Report1.txt
Loading file Report1.txt
True
Printing Attributes :  ['Age', 'HyperTension', 'Complication']
Printing Decision :  Delivery


In [26]:
from pyspark.sql.functions import monotonically_increasing_id
def convertTableToSparkDataframe():
    df = spark.createDataFrame(table_content , table_header)
    #res = df.withColumn("id", monotonically_increasing_id()+1)
    print('Do you want to display the table (Y/N) ')
    choice = input("> ")
    if(choice=='Y' or choice == 'y'):
        res.show()
    return res

In [37]:
table_header.append("id")

In [38]:
res=convertTableToSparkDataframe()

Do you want to display the table (Y/N) 
> y
+---+------------+------------+--------+---+
|Age|HyperTension|Complication|Delivery| id|
+---+------------+------------+--------+---+
| 26|          no|        none| preterm|  1|
| 26|         yes|     obesity| preterm|  2|
| 34|          no|     obesity|fullterm|  3|
| 34|         yes|     obesity|fullterm|  4|
| 28|         yes|     obesity|fullterm|  5|
| 28|         yes|        none| preterm|  6|
+---+------------+------------+--------+---+



In [36]:
table_content

[['26', 'no', 'none', 'preterm', 3],
 ['26', 'yes', 'obesity', 'preterm', 4],
 ['34', 'no', 'obesity', 'fullterm', 5],
 ['34', 'yes', 'obesity', 'fullterm', 6],
 ['28', 'yes', 'obesity', 'fullterm', 7],
 ['28', 'yes', 'none', 'preterm', 8]]

In [19]:
table_header

['Age', 'HyperTension', 'Complication', 'Delivery']

['Age', 'HyperTension', 'Complication', 'Delivery', 'ID']

In [7]:
res.show()

+------------+-----------+------------+-----------+-----------+---+
|sepal_length|sepal_width|petal_length|petal_width|      class| id|
+------------+-----------+------------+-----------+-----------+---+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|  1|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|  2|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|  3|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|  4|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|  5|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|  6|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|  7|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|  8|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|  9|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa| 10|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa| 11|
|         4.8|        3.4|         1.6|        0

In [8]:
def is_numerical(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [9]:

def creatingBlocks(blocks,data_type,data_type_num,table_header,res):
    for col in table_header:
        new_df1=res.select(col).distinct()
        b={}
        count=0
        if col in data_type_num.keys():
            count=data_type_num[col]
        for r in new_df1.head(new_df1.count()):
            #print(r[0])
            list1 = res.filter(res[col]==r[0]).select('id').collect()
            list1_array =[int(i.id) for i in list1]
            list1_array=list(set(list1_array))
            b[r[0]]= list1_array
            if(is_numerical(r[0])):
                count+=len(list1_array)
        data_type_num[col]=count

        if col in blocks:
            blocks[col].append(b)
        else:
            blocks[col]=b
    print('Do you want to display the Blocks (Y/N) ')
    choice = input("> ")
    if(choice=='Y' or choice == 'y'):
        print(blocks)
    return blocks,data_type_num,data_type

In [10]:
data_type_num ={}
data_type = {}
blocks ={}
blocks,data_type_num,data_type=creatingBlocks(blocks,data_type,data_type_num,table_header,res)

Do you want to display the Blocks (Y/N) 
> 


In [11]:
def openFileToWrite(f_name):
    temp_fileHandler= open('Log.txt','w')
    temp_fileHandler.write('Starting file to write \n')
    return temp_fileHandler

In [12]:
def writeBlockInFile(file,columns,blocks):
    file.write('\n Dictionary\n') 
    for attr in columns:
        file.write('Column/Attribute : ')
        file.write(attr) 
        file.write('\n')
        for val in blocks[attr]:
            file.write(val)
            file.write(': [ ')
            tem=blocks[attr][val]
            for v in tem:
                file.write(str(v))
                file.write(' ')
            file.write(']\n')

In [13]:
def close_file(file):
    file.write('\n ------ Closing File ------\n')
    file.close() 

In [14]:
#writing the block values in a file
file1=openFileToWrite('Block_Output.txt')
writeBlockInFile(file1,columns,blocks)

In [15]:
#
def processingDataForMissingValues():
    list_missing=[]
    for attr in columns:
        list_missing=[]
        #missing value *
        if "*" in blocks[attr]:
            list_missing = blocks[attr].pop('*',None)
            if(len(list_missing)>0):
                for val in blocks[attr]:
                    blocks[attr][val] = list(set(blocks[attr][val]).union(set(list_missing)))
        #missing value -            
        if "-" in blocks[attr]:
            list_missing = blocks[attr].pop('-',None)
            if(len(list_missing)>0):
                for num in list_missing:
                    #print(decision_rows[num])
                    for val in blocks[attr]:
                        if(len(list(set(set(blocks[Decision][decision_rows[num]]).intersection(set(set(blocks[attr][val]))))))>0):
                            Temp_arr=[]
                            Temp_arr.append(num)
                            blocks[attr][val] = list(set(blocks[attr][val]).union(set(Temp_arr)))
        #missing value ?
        if "?" in blocks[attr]:
            blocks[attr].pop('?',None)
        #print(blocks)
    return blocks

In [16]:
blocks=processingDataForMissingValues()

In [17]:
#blocks

In [18]:
# writing the updated values in file
writeBlockInFile(file1,columns,blocks)

In [19]:
# remove the restriction
def ClassifyTableData():
    column_count_limit = column_count*0.75
    for col in table_header:
        if(data_type_num[col]>column_count_limit):
            data_type[col]="Numerical"
        else:
            data_type[col]="Discrete"
    return data_type

In [20]:
data_type=ClassifyTableData()

In [21]:
print(data_type)

{'sepal_width': 'Numerical', 'class': 'Discrete', 'petal_length': 'Numerical', 'sepal_length': 'Numerical', 'petal_width': 'Numerical'}


In [22]:
def add_Cases(formatedCut,colName,values):
    if formatedCut in blocks[colName]:
        blocks[colName][formatedCut]=list(set(blocks[colName][formatedCut]).union(set(values)))
    else:
        blocks[colName][formatedCut]=values

In [23]:
def convert_NumericalToConceptual():
    for colName in columns:
        if(data_type[colName]=='Numerical'):
            tempArray = sorted(list(blocks[colName].keys()))
            list_cutpoints = []
            for cutpoint_i in range(len(tempArray)-1):
                list_cutpoints.append((float(tempArray[cutpoint_i])+float(tempArray[cutpoint_i+1]))/2)
            for val in tempArray:
                for cut_point in list_cutpoints:
                    values = blocks[colName][val]
                    if(float(val)>cut_point):
                        formatedCut = " > {0:.2f} ".format(cut_point)
                    elif (float(val)<=cut_point):
                        formatedCut = " <= {0:.2f} ".format(cut_point)
                    add_Cases(formatedCut,colName,values)

            for val in tempArray:
                blocks[colName].pop(val,None)

In [24]:
convert_NumericalToConceptual()

In [25]:
#blocks

In [26]:
file1.write('\n Updated Dictionary converting Numerical Attributes to Conceptual\n') 
writeBlockInFile(file1,columns,blocks)

In [58]:
from RuleClass import RuleClass
def print_rule(curRule,sr_no):
        print('Rule ',sr_no)
        #print(curRule.attributes, curRule.attributes_value)
        #k=zip(curRule.attributes, curRule.attributes_value)
        #print(list(k))
        rule_str=''
        is_first=True
        for i in range(len(curRule.attributes)):
            if(is_first==False):
                rule_str = rule_str +' & '
            rule_str=rule_str +'('+curRule.attributes[i]+', '+curRule.attributes_value[i]+")"
            is_first=False
        rule_str = rule_str +' --> ('+Decision+', '+curRule.Decision+")"
        print(rule_str,',')
        #print('Decision ',curRule.Decision)
        #print('Decision Covered ',curRule.decision_cover)
        #print('InCorrect Cover ',curRule.false_cover)
        print('Strength: ',curRule.strength,',')
        print('Specificity: ',curRule.specificity,',')
        print('Support: ',curRule.support,',')
        print('Conditional Probablity: ',curRule.conditionalProbablity,'.')
        print('----------------------------------------------------------------------------------')

In [28]:
def print_rule_without_detials(curRule,sr_no):
        print('Rule ',sr_no)
        #print(curRule.attributes, curRule.attributes_value)
        #k=zip(curRule.attributes, curRule.attributes_value)
        #print(list(k))
        rule_str=''
        is_first=True
        for i in range(len(curRule.attributes)):
            if(is_first==False):
                rule_str = rule_str +' & '
            rule_str=rule_str +'('+curRule.attributes[i]+', '+curRule.attributes_value[i]+")"
            is_first=False
        rule_str = rule_str +' --> ('+Decision+', '+curRule.Decision+")"
        print(rule_str)
        #print('Decision ',curRule.Decision)
        print('--------------------------------------------------------------------------------------------------')

In [38]:
min_rule_Length=1
max_rule_Length=1
min_Strength=5
min_prob=1
print('Default rule specification, (Please enter c or C, if you want to change any of the below condition)')
print('Minimum length of rule ',min_rule_Length)
print('Maximum length of rule ',max_rule_Length)
print('Minimum Strength of rule ',min_Strength)
print('Minimum Conditional Probablity ',min_prob)
choice = input("> ")
if(choice=='c' or choice == 'C'):
    print('Enter minimum length of rule ',min_rule_Length)
    min_rule_Length= input("> ")
    print('Enter maximum length of rule ',max_rule_Length)
    max_rule_Length= input("> ")
    print('Enter minimum Strength of rule ',min_Strength)
    min_Strength= input("> ")
    print('Enter minimum Conditional Probablity ',min_prob)
    min_rule_Length= input("> ")

Default rule specification, (Please enter c or C, if you want to change any of the below condition)
Minimum length of rule  1
Maximum length of rule  1
Minimum Strength of rule  5
Minimum Conditional Probablity  1
> 


In [39]:
possibleRules = []
rules = []
all_ID = res.select('id').collect()
all_ID =[int(i.id) for i in all_ID]

In [40]:
def compute_rules(min_rule_Length,max_rule_Length,min_Strength,min_prob):
    temp_Decision=''
    DecisionCovered = all_ID
    #possible Rules
    for j in range(min_rule_Length,max_rule_Length+1):   
        for idx, attributeSet in enumerate(itertools.combinations(columns, j)):
            #print(idx)
            print(attributeSet)
            valueSet = [[val for val in blocks[attr].keys() if val not in ["*", "?", "-"]] for attr in attributeSet]
            #print('Value Set ',valueSet)
            for combination in itertools.product(*valueSet):
                k=zip(attributeSet, combination)
                #print(attributeSet,len(combination))
                #print('Printing zip',list(k))
                temp=res
                temp_ID = all_ID
                for ri in range(len(attributeSet)):
                    templist123= blocks[attributeSet[ri]][combination[ri]]              
                    temp_ID=list(set(temp_ID).intersection(set(templist123)))
                    #temp = temp.filter(temp[attributeSet[ri]]==combination[ri])
                #list_arr= temp.select('id').collect()
                strength=len(temp_ID)
                if(strength<min_Strength):
                    continue
                #print(temp_ID)
                max_covered =0
                for key in blocks[Decision]:
                    DecisionCovered=list(set(temp_ID).intersection(set(blocks[Decision][key])))
                    if(max_covered<len(DecisionCovered)):
                        max_covered=len(DecisionCovered)
                        temp_Decision=key
                DecisionCovered=list(set(temp_ID).intersection(set(blocks[Decision][temp_Decision]))) 
                cond_prob=float(len(DecisionCovered))/float(strength)
                if(cond_prob<min_prob ):
                    continue
                inCorrectCover=list(set(temp_ID)-set(DecisionCovered))
                #list1_array_temp =[int(i.id) for i in list_arr]
                #print('Covered by Rule',list1_array_temp, temp_ID)
                temp_rule=RuleClass(list(attributeSet),list(combination),temp_ID,temp_Decision,DecisionCovered,inCorrectCover)
                possibleRules.append(temp_rule)
    return possibleRules

In [41]:
possibleRules=compute_rules(min_rule_Length,max_rule_Length,min_Strength,min_prob)

('sepal_length',)
('sepal_width',)
('petal_length',)
('petal_width',)


In [42]:
def Display_Rules_SortStrength(possibleRules):
    possibleRules = sorted(possibleRules, key=lambda each_rule: each_rule.strength,reverse=True)
    #for each_rule in possibleRules:
        #print_rule(each_rule)
    return possibleRules

In [43]:
def Display_Rules_SortConditionalProbablity(possibleRules):
    possibleRules = sorted(possibleRules, key=lambda each_rule: each_rule.strength,reverse=True)
    possibleRules = sorted(possibleRules, key=lambda each_rule: each_rule.conditionalProbablity,reverse=True)
    #for each_rule in possibleRules:
        #if(float(min_prob)>float(each_rule.conditionalProbablity)):
            #break
        #print_rule(each_rule)
    return possibleRules

In [44]:
print("Number of rules generated",len(possibleRules))
print('Enter \n 1. Sort rules by Conditional Probbality \n 2. Sort rules by Rule Strength \n By default 2 is selected')
choice = input("> ")
if(choice==1):
    possibleRules1=Display_Rules_SortStrength(possibleRules)
else:
    possibleRules1=Display_Rules_SortConditionalProbablity(possibleRules)

Number of rules generated 41
Enter 
 1. Sort rules by Conditional Probbality 
 2. Sort rules by Rule Strength 
 By default 2 is selected
> 


In [45]:
sr_no=1
for each_rule in possibleRules1:
    print_rule_without_detials(each_rule,sr_no)
    sr_no+=1

Rule  1
(petal_length,  <= 2.45 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  2
(petal_width,  <= 0.80 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  3
(petal_width,  <= 0.55 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  4
(petal_length,  <= 1.80 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  5
(petal_width,  <= 0.45 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  6
(petal_length,  <= 1.65 ) --> (class, Iris-setosa)
--------------------------------------------------------------------------------------------------
Rule  7
(petal_width,  <= 0.35 ) --> (class, Iris-setos

In [59]:
sr_no=0
for each_rule in possibleRules1:
    sr_no=sr_no+1
    print_rule(each_rule,sr_no)

Rule  1
(petal_length,  <= 2.45 ) --> (class, Iris-setosa) ,
Strength:  50 ,
Specificity:  1 ,
Support:  50 ,
Conditional Probablity:  1.0 .
----------------------------------------------------------------------------------
Rule  2
(petal_width,  <= 0.80 ) --> (class, Iris-setosa) ,
Strength:  50 ,
Specificity:  1 ,
Support:  50 ,
Conditional Probablity:  1.0 .
----------------------------------------------------------------------------------
Rule  3
(petal_width,  <= 0.55 ) --> (class, Iris-setosa) ,
Strength:  49 ,
Specificity:  1 ,
Support:  49 ,
Conditional Probablity:  1.0 .
----------------------------------------------------------------------------------
Rule  4
(petal_length,  <= 1.80 ) --> (class, Iris-setosa) ,
Strength:  48 ,
Specificity:  1 ,
Support:  48 ,
Conditional Probablity:  1.0 .
----------------------------------------------------------------------------------
Rule  5
(petal_width,  <= 0.45 ) --> (class, Iris-setosa) ,
Strength:  48 ,
Specificity:  1 ,
Support:  48

In [47]:
out_fileHandler= open('output1.txt','w')
sr_no=0
for curRule in possibleRules1:
    sr_no=sr_no+1
    #out_fileHandler.write(str (sr_no))
    #out_fileHandler.write('. ')
    rule_str=''
    is_first=True
    for i in range(len(curRule.attributes)):
        if(is_first==False):
            rule_str = rule_str +' & '
        rule_str=rule_str +'('+curRule.attributes[i]+', '+curRule.attributes_value[i]+")"
        is_first=False
    rule_str = rule_str +' --> ('+Decision+', '+curRule.Decision+")"
    out_fileHandler.write(rule_str)
    out_fileHandler.write('\n')
    #print('Decision ',curRule.Decision)
    #out_fileHandler.write('--------------------------------------------------------------------------------------------------')
    #out_fileHandler.write('\n')
    if(sr_no==1000):
        break
out_fileHandler.write('\n ------ Closing File ------\n')
out_fileHandler.close()

In [43]:
sr_no

5839